In [68]:
import pandas as pd

# Load the data
df = pd.read_csv("./data/2024_flavors_of_cacoa.tsv", sep='\t')


df.head()


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",68%,"3- B,S,C","milk brownie, macadamia,chewy",3.50
4,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",80%,"3- B,S,C","mildly bitter, basic cocoa, fatty",3.25


In [69]:
# Convert "Cocoa Percent" from string 76% to 0.76
df["Cocoa Percent"] = df["Cocoa Percent"].str.replace("%", "").astype(float) / 100


df.head()


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",0.76,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",0.76,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",0.76,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",0.68,"3- B,S,C","milk brownie, macadamia,chewy",3.50
4,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",0.80,"3- B,S,C","mildly bitter, basic cocoa, fatty",3.25


In [70]:
# Process 'Ingredients'
print("Number of Columns Before Step 4:", len(df.columns))

Number of Columns Before Step 4: 10


In [71]:
ingredient_columns = ['B', 'C', 'L', 'S', 'S*', 'Sa', 'V']
def split_ingredients(ingredient_str):
    if pd.isna(ingredient_str):
        return [0] * len(ingredient_columns) + [0]
    parts = ingredient_str.split('- ')
    count = int(parts[0]) if parts[0].isdigit() else 0
    ingredients = set(map(str.strip, parts[1].split(','))) if len(parts) > 1 else set()
    return [1 if col in ingredients else 0 for col in ingredient_columns] + [count]

ingredients_df = df["Ingredients"].apply(split_ingredients).apply(pd.Series)
ingredients_df.columns = ingredient_columns + ["ingredient_count"]
ingredients_df = ingredients_df.astype(int)
df = pd.concat([df, ingredients_df], axis=1)

# Display dataframe head and column count after Step 4
print("DataFrame Head After Step 4:")
print(df.head())
print("\nNumber of Columns After Step 4:", len(df.columns))


DataFrame Head After Step 4:
    REF Company (Manufacturer) Company Location  Review Date  \
0  2454                   5150           U.S.A.         2019   
1  2458                   5150           U.S.A.         2019   
2  2454                   5150           U.S.A.         2019   
3  2542                   5150           U.S.A.         2021   
4  2546                   5150           U.S.A.         2021   

  Country of Bean Origin Specific Bean Origin or Bar Name  Cocoa Percent  \
0             Madagascar           Bejofo Estate, batch 1           0.76   
1     Dominican Republic                  Zorzal, batch 1           0.76   
2               Tanzania            Kokoa Kamili, batch 1           0.76   
3                  India                Anamalai, batch 1           0.68   
4                 Uganda         Semuliki Forest, batch 1           0.80   

  Ingredients     Most Memorable Characteristics  Rating  B  C  L  S  S*  Sa  \
0    3- B,S,C       cocoa, blackberry, full body 

In [72]:
# Process 'Most Memorable Characteristics'
characteristics = df["Most Memorable Characteristics"].str.get_dummies(sep=", ")
char_counts = characteristics.sum().sort_values(ascending=False)
common_chars = char_counts.head(71).index.tolist()  # Ensure exactly 71 columns are selected
print("\nNumber of 'Most Memorable Characteristics' Columns Being Added:", len(common_chars))
print("Columns Being Added:", common_chars)


Number of 'Most Memorable Characteristics' Columns Being Added: 71
Columns Being Added: ['sweet', 'nutty', 'cocoa', 'roasty', 'earthy', 'creamy', 'fatty', 'sandy', 'floral', 'sour', 'spicy', 'intense', 'fruit', 'molasses', 'woody', 'vanilla', 'sticky', 'dried fruit', 'rich', 'coffee', 'tart', 'grassy', 'caramel', 'bitter', 'spice', 'gritty', 'dry', 'fruity', 'rich cocoa', 'dairy', 'oily', 'brownie', 'complex', 'banana', 'acidic', 'smokey', 'pungent', 'astringent', 'off', 'cherry', 'vegetal', 'tobacco', 'honey', 'smoke', 'metallic', 'nut', 'melon', 'mild fruit', 'rubbery', 'dirty', 'smooth', 'hammy', 'mild bitter', 'strawberry', 'off notes', 'bold', 'red berry', 'coarse', 'muted', 'burnt', 'citrus', 'rubber', 'basic cocoa', 'black pepper', 'marshmallow', 'green', 'chemical', 'bland', 'tangy', 'nuts', 'raisins']


In [73]:
# Ensure only 71 unique columns are added and prevent duplication
characteristics = characteristics[common_chars]
characteristics = characteristics.loc[:, ~characteristics.columns.duplicated()]
df = pd.concat([df, characteristics], axis=1)


In [74]:
#  Final Cleanup
df.drop(columns=["Ingredients", "Most Memorable Characteristics"], inplace=True, errors='ignore')

In [75]:
# Identify and drop the extra column if present
if len(df.columns) > 86:
    extra_columns = df.columns[86:]
    print("\nExtra Column Detected and Removed:", extra_columns.tolist())
    df.drop(columns=extra_columns, inplace=True)



Extra Column Detected and Removed: ['raisins']


In [76]:
#  Verifying Final Column Count
print("\nExpected Number of Columns: 86")
print("Actual Number of Columns:", len(df.columns))
print("Column Names:", df.columns.tolist())


Expected Number of Columns: 86
Actual Number of Columns: 86
Column Names: ['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date', 'Country of Bean Origin', 'Specific Bean Origin or Bar Name', 'Cocoa Percent', 'Rating', 'B', 'C', 'L', 'S', 'S*', 'Sa', 'V', 'ingredient_count', 'sweet', 'nutty', 'cocoa', 'roasty', 'earthy', 'creamy', 'fatty', 'sandy', 'floral', 'sour', 'spicy', 'intense', 'fruit', 'molasses', 'woody', 'vanilla', 'sticky', 'dried fruit', 'rich', 'coffee', 'tart', 'grassy', 'caramel', 'bitter', 'spice', 'gritty', 'dry', 'fruity', 'rich cocoa', 'dairy', 'oily', 'brownie', 'complex', 'banana', 'acidic', 'smokey', 'pungent', 'astringent', 'off', 'cherry', 'vegetal', 'tobacco', 'honey', 'smoke', 'metallic', 'nut', 'melon', 'mild fruit', 'rubbery', 'dirty', 'smooth', 'hammy', 'mild bitter', 'strawberry', 'off notes', 'bold', 'red berry', 'coarse', 'muted', 'burnt', 'citrus', 'rubber', 'basic cocoa', 'black pepper', 'marshmallow', 'green', 'chemical', 'bland', 'tang

In [77]:
# Saving the cleaned data
df.to_csv("data/cleaned_data_2025_flavors_of_cacao.csv", index=False)


In [78]:
# Displaying final confirmation
print("Data cleaning complete! Saved as 'data/cleaned_data_2025_flavors_of_cacao.csv'.")

Data cleaning complete! Saved as 'data/cleaned_data_2025_flavors_of_cacao.csv'.


In [80]:
# Extracting and save reduced dataset
columns_to_keep = ["Review Date", "Country of Bean Origin", "Cocoa Percent", "Rating"]
df_reduced = df[columns_to_keep]
df_reduced.to_csv("data/data_reduced_2025_flavors_of_cacao.csv", index=False)
df_reduced.to_json("data/data_reduced_2025_flavors_of_cacao.json", orient="records")

In [82]:
# Filtering data based on conditions
filtered_df = df[
    (df["Rating"] >= 3.25) &
    (df["Cocoa Percent"] >= 0.65) & (df["Cocoa Percent"] <= 0.75) &
    (df["Review Date"] >= 2018) & (df["Review Date"] <= 2021) &
    ((df["fatty"] == 1) | (df["earthy"] == 1) | (df["roasty"] == 1))
]


In [83]:
# Saving the filtered dataset
filtered_df.to_csv("data/data_filtered_2025_flavors_of_cacao.csv", index=False)
filtered_df.to_json("data/data_filtered_2025_flavors_of_cacao.json", orient="records")

In [84]:
# Step 13: Display final confirmation
print("Data extraction, transformation, and export complete!")

Data extraction, transformation, and export complete!
